In [1]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
import logging
import os
import time
import warnings

warnings.filterwarnings('ignore')

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(message)s')

pd.set_option('display.max_rows', None)


def greedy_arima(df, column, p_range=(28, 50), d_range=(0, 3), q_range=(28, 50), test_size=0.2):
    start_time = time.time()  # Start timing the function
    data = df[column].dropna()

    # Train-test split
    train_data, test_data = train_test_split(data, test_size=test_size, shuffle=False)
    
    best_score, best_params = float('inf'), None

    # Grid search for best p, d, q
    for p in range(*p_range):
        try:
            model = ARIMA(train_data, order=(p, 0, 0)).fit()
            predictions = model.predict(start=len(train_data), end=len(train_data) + len(test_data) - 1)
            mae = mean_absolute_error(test_data, predictions)
            if mae < best_score:
                best_score, best_params = mae, (p, 0, 0)
        except Exception:
            continue

    best_p, _, best_q = best_params
    for d in range(*d_range):
        try:
            model = ARIMA(train_data, order=(best_p, d, best_q)).fit()
            predictions = model.predict(start=len(train_data), end=len(train_data) + len(test_data) - 1)
            mae = mean_absolute_error(test_data, predictions)
            if mae < best_score:
                best_score, best_params = mae, (best_p, d, best_q)
        except Exception:
            continue

    best_p, best_d, _ = best_params
    for q in range(*q_range):
        try:
            model = ARIMA(train_data, order=(best_p, best_d, q)).fit()
            predictions = model.predict(start=len(train_data), end=len(train_data) + len(test_data) - 1)
            mae = mean_absolute_error(test_data, predictions)
            if mae < best_score:
                best_score, best_params = mae, (best_p, best_d, q)
        except Exception:
            continue

    end_time = time.time()
    logging.info(f"Best Parameters: {best_params} with MAE: {best_score}")
    logging.info(f"Runtime for greedy_arima: {end_time - start_time:.2f} seconds")
    return best_params, best_score


def train_arima_model(df, column, params):
    start_time = time.time()
    model = ARIMA(df[column], order=params).fit()
    end_time = time.time()
    logging.info(f"Runtime for train_arima_model: {end_time - start_time:.2f} seconds")
    return model


def main():
    overall_start_time = time.time()

    for group_id, (group_name, commodities) in COMMODITY_GROUPS.items():
        logging.info(f"Processing commodity group: {group_name}")
        
        for commodity_id, (commodity_name, csv_path) in commodities.items():
            logging.info(f"Processing commodity: {commodity_name}")
            
            if not os.path.exists(csv_path):
                logging.error(f"File not found: {csv_path}")
                continue

            df = pd.read_csv(csv_path, index_col='Date', parse_dates=True)
            logging.info(f"Data loaded successfully for {commodity_name}.")

            for column in ["High", "Low"]:
                if column not in df.columns:
                    logging.warning(f"Column '{column}' not found in the dataset. Skipping analysis for this column.")
                    continue

                logging.info(f"Analyzing {column} prices for {commodity_name}.")
                params, mae = greedy_arima(df, column)
                if params:
                    logging.info(f"Optimal parameters for {column} in {commodity_name}: {params} with MAE: {mae}")
                    model = train_arima_model(df, column, params)
                else:
                    logging.warning(f"No valid parameters found for {column} in {commodity_name}.")

    overall_end_time = time.time()
    logging.info(f"Total runtime for the script: {overall_end_time - overall_start_time:.2f} seconds")


if __name__ == "__main__":    
    COMMODITY_GROUPS = {
        # 1: ("Rice", {
        #     1: ('Regular-Milled Rice', '../csv/rice/regular_milled_rice.csv'),
        #     2: ('Well-Milled Rice', '../csv/rice/well_milled_rice.csv'),
        #     3: ('Premium Rice', '../csv/rice/premium_rice.csv'),
        #     4: ('Special Rice', '../csv/rice/special_rice.csv'),
        # }),
        2: ("Meat", {
            # 1: ('Beef Brisket', '../csv/meat/beef_brisket.csv'),
            # 2: ('Beef Rump', '../csv/meat/beef_rump.csv'),
            3: ('Whole Chicken', '../csv/meat/whole_chicken.csv'),
            4: ('Pork Belly', '../csv/meat/pork_belly.csv'),
            5: ('Pork Ham', '../csv/meat/pork_kasim.csv'),
        }),
        3: ("Fish", {
            1: ('Alumahan', '../csv/fish/alumahan.csv'),
            2: ('Bangus', '../csv/fish/bangus.csv'),
            3: ('Galunggong', '../csv/fish/galunggong.csv'),
            4: ('Tilapia', '../csv/fish/tilapia.csv'),
        }),
        4: ("Fruits", {
            # 1: ('Banana (Lakatan)', '../csv/fruits/banana_lakatan.csv'),
            2: ('Calamansi', '../csv/fruits/calamansi.csv'),
            3: ('Mango', '../csv/fruits/mango.csv'),
            4: ('Papaya', '../csv/fruits/papaya.csv'),
        }),
        5: ("Vegetables", {
            1: ('Cabbage', '../csv/vegetables/cabbage.csv'),
            2: ('Carrots', '../csv/vegetables/carrots.csv'),
            3: ('Eggplant', '../csv/vegetables/eggplant.csv'),
            4: ('Tomato', '../csv/vegetables/tomato.csv'),
            5: ('Potato', '../csv/vegetables/white_potato.csv'),
        }),
        6: ("Spices", {
            1: ('Garlic', '../csv/spices/garlic.csv'),
            # 2: ('Red Onion', '../csv/spices/red_onion.csv'),
        })
    }
    
    main()

2024-12-16 13:19:08,904 - Processing commodity group: Meat
2024-12-16 13:19:08,905 - Processing commodity: Whole Chicken
2024-12-16 13:19:08,915 - Data loaded successfully for Whole Chicken.
2024-12-16 13:19:08,918 - Analyzing High prices for Whole Chicken.
2024-12-16 13:40:56,762 - Best Parameters: (39, 0, 46) with MAE: 6.136442588268983
2024-12-16 13:40:56,763 - Runtime for greedy_arima: 1307.84 seconds
2024-12-16 13:40:56,776 - Optimal parameters for High in Whole Chicken: (39, 0, 46) with MAE: 6.136442588268983
2024-12-16 13:41:56,812 - Runtime for train_arima_model: 60.04 seconds
2024-12-16 13:41:56,812 - Analyzing Low prices for Whole Chicken.
2024-12-16 14:03:16,748 - Best Parameters: (39, 0, 0) with MAE: 13.493451150180224
2024-12-16 14:03:16,749 - Runtime for greedy_arima: 1279.93 seconds
2024-12-16 14:03:16,764 - Optimal parameters for Low in Whole Chicken: (39, 0, 0) with MAE: 13.493451150180224
2024-12-16 14:03:37,078 - Runtime for train_arima_model: 20.31 seconds
2024-12-1